# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686600


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:24,  2.83s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<01:02,  2.22s/it]

Rendering models:  19%|█▉        | 6/32 [00:07<00:41,  1.58s/it]

Rendering models:  22%|██▏       | 7/32 [00:07<00:28,  1.15s/it]

...

Rendering models:  84%|████████▍ | 27/32 [00:09<00:00,  7.85it/s]

Rendering models:  88%|████████▊ | 28/32 [00:09<00:00,  7.88it/s]

Rendering models:  91%|█████████ | 29/32 [00:09<00:00,  7.92it/s]

Rendering models:  97%|█████████▋| 31/32 [00:10<00:00,  8.39it/s]

BridieK                               0.000322
not-logged-in-20a275b53abec8db9f41    0.372760
Dynamatt                              0.000149
not-logged-in-f5ac928c4dcd4e9cb619    0.000226
not-logged-in-a7f3c0eb6d81d69e04a2    0.000134
kjkavene                              0.000105
acohen12                              0.000103
kcarte13                              0.000473
FelipedaFonseca                       0.000302
not-logged-in-9a45911eb71184fe7c7f    0.000591
not-logged-in-5f30b2bb9b7f9d49da41    0.010528
Arlo-M                                0.021212
not-logged-in-f6483c913f721ef6ba74    0.000098
crush202020                           0.000166
clairedeu                             0.000121
haydenmanninen                        0.000077
Susymuller                            0.000249
rhondacard                            0.001159
Lavadude                              0.005070
not-logged-in-d1a2200e729ec53382b7    0.000782
Mr_DMan                               0.000296
ykuzmits     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())